In [2]:
%env https_proxy=http://proxy.valeria.science:3128

env: http_proxy=http://proxy.valeria.science:3128


In [2]:
import sys
import moviepy
import pandas as pd 
from moviepy.editor import *
from datetime import  datetime
#video_file= "../sante_veau/videos/2022_02_02/N862A6_ch1_main_20220125000000_20220125003000.mp4"


In [3]:
interesting_sections=pd.read_csv("../louves/visits_point.csv", sep=";")
interesting_sections=interesting_sections.loc[interesting_sections['Duration'] !=0]
interesting_sections=interesting_sections[["date", "station", "Duration", "calfNumber"]]
interesting_sections["station"] =interesting_sections["station"].replace([1,2],[2,1])

interesting_sections["start visit dateTime"]= pd.to_datetime(interesting_sections["date"])
interesting_sections["end visit dateTime"]= interesting_sections["start visit dateTime"] + pd.to_timedelta(interesting_sections['Duration']+30, unit='s')- pd.to_timedelta(6, unit='h')
interesting_sections["start visit dateTime"]= pd.to_datetime(interesting_sections["date"])- pd.to_timedelta(60, unit='s') - pd.to_timedelta(6, unit='h')#.agg(' '.join, axis=1))#.dt.time
interesting_sections.to_csv("nonull_duration.csv", index = None,  header=True)

interesting_sections.head()
#pd.to_datetime(interesting_sections[["date", "end visit"]].agg(' '.join, axis=1))#.dt.time



,date,station,Duration,calfNumber,start visit dateTime,end visit dateTime
14,2022-01-30T15:13,1,586,6700,2022-01-30 09:12:00,2022-01-30 09:23:16
25,2022-01-30T14:42,1,470,6335,2022-01-30 08:41:00,2022-01-30 08:50:20
26,2022-01-30T14:34,2,160,6685,2022-01-30 08:33:00,2022-01-30 08:37:10
31,2022-01-30T14:12,2,96,6314,2022-01-30 08:11:00,2022-01-30 08:14:06
34,2022-01-30T14:05,2,127,6692,2022-01-30 08:04:00,2022-01-30 08:07:37


In [4]:
import os
path ="../../sante_veau/videos/2022_02_02/"
availabe_videos=pd.DataFrame(columns=["names"])

#we shall store all the file names in this list
filelist = []
for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        filelist.append(os.path.join(root,file))
#print all the file names
for name in filelist:
    #print(name)
    availabe_videos = availabe_videos.append({'names':name}, ignore_index=True)

    

    ###"remplacer par toutes les vidéos présentent dans le dossier", à trier impérativement pour les portions partagées
    ### 
    availabe_videos["start time"]=[(name.split(".mp4")[0].split("_")[-2])[:-6]+" "+(name.split(".mp4")[0].split("_")[-2])[-6:-4]+":"+(name.split(".mp4")[0].split("_")[-2])[-4:-2]+":"+(name.split(".mp4")[0].split("_")[-2])[-2:] for name in  availabe_videos["names"].values]
    availabe_videos["end time"]=[(name.split(".mp4")[0].split("_")[-1])[:-6]+" "+(name.split(".mp4")[0].split("_")[-1])[-6:-4]+":"+(name.split(".mp4")[0].split("_")[-1])[-4:-2]+":"+(name.split(".mp4")[0].split("_")[-1])[-2:]  for name in  availabe_videos["names"].values]
    availabe_videos["start time"] = pd.to_datetime(availabe_videos['start time'],infer_datetime_format=True) # format ='%y%m%d %H:%M:%S')
    availabe_videos["end time"] = pd.to_datetime(availabe_videos['end time'],infer_datetime_format=True) 
#availabe_videos.head()


# Extracting interesting sections in videos


In [ ]:
import  copy
def crop_video(video_name, new_name, start, end):
    video = VideoFileClip(video_name).subclip(start,end)
    return video
    

def search_interesting_section (interesting_section):
    for video_idx, availabe_video in availabe_videos.iterrows():
        if "_ch"+str(interesting_section["station"])+"_" in availabe_video["names"] :
            #print("dispo",availabe_video["names"])
            if interesting_section["start visit dateTime"]>= availabe_video["start time"] and interesting_section["start visit dateTime"] <= availabe_video["end time"]:
                #print(availabe_video["end time"], interesting_section["start visit dateTime"], interesting_section["end visit dateTime"], availabe_video["start time"])
                #print((availabe_video["start time"] - interesting_section["start visit dateTime"]). total_seconds() )
                print("hi",availabe_video["names"])
                
                print(interesting_section["start visit dateTime"]>= availabe_video["start time"], interesting_section["start visit dateTime"], availabe_video["start time"])

                start=  (interesting_section["start visit dateTime"] - availabe_video["start time"]).total_seconds()
                end=  (interesting_section["end visit dateTime"] - availabe_video["start time"]).total_seconds()

                rest = -(availabe_video["end time"] - interesting_section["end visit dateTime"]).total_seconds()
                end_video= (availabe_video["end time"] - availabe_video["start time"]).total_seconds()
                print("**",start,end, "the rest is", rest)
                if rest>0:
                    video = crop_video(availabe_video["names"], interesting_section["start visit dateTime"].strftime("%m%d%Y%H%M%S")+"__to__"+interesting_section["end visit dateTime"].strftime("%m%d%Y%H%M%S"), start, end_video )
                    print("we should take in another video")
                    rest_interesting_section=copy.deepcopy(interesting_section)
                    rest_interesting_section["start visit dateTime"]= availabe_video["end time"]+pd.to_timedelta(1, unit='s')
                    rest_video = search_interesting_section (rest_interesting_section)
                    final = concatenate_videoclips([video, rest_video])
                    return final

                else:
                    print("we crop")
                    video = crop_video(availabe_video["names"], interesting_section["start visit dateTime"].strftime("%m%d%Y%H%M%S")+"__to__"+interesting_section["end visit dateTime"].strftime("%m%d%Y%H%M%S"), start, end )
                    return video
        
    return None

        
path ="videos/"
filelist = []
for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        filelist.append(os.path.join(root,file))
print(filelist)
for section_idx, interesting_section in interesting_sections.iterrows():
    #enlever le _ a coté de ch
    new_name="vealnum_"+str(interesting_section["calfNumber"])+"_ch"+str(interesting_section["station"])+"_from_"+interesting_section["start visit dateTime"].strftime("%m%d%Y%H%M%S")+"__to__"+interesting_section["end visit dateTime"].strftime("%m%d%Y%H%M%S")
    if "videos/"+new_name+".mp4" not in filelist:
        #print("videos/"+new_name)
        video= search_interesting_section(interesting_section)
        if video!=None :
            video.write_videofile("videos/"+new_name+".mp4",fps=2)
            print("******created","videos/"+new_name)


['videos/vealnum_6686_ch2_from_01262022070700__to__01262022070844.mp4', 'videos/vealnum_6692_ch2_from_01272022082300__to__01272022083318.mp4', 'videos/vealnum_6677_ch2_from_01252022092800__to__01252022093221.mp4', 'videos/vealnum_6324_ch2_from_01272022080800__to__01272022080937.mp4', 'videos/vealnum_6338_ch1_from_01262022033500__to__01262022034417.mp4', 'videos/vealnum_6335_ch1_from_01262022023500__to__01262022024444.mp4', 'videos/vealnum_6323_ch2_from_01262022080100__to__01262022080745.mp4', 'videos/vealnum_6686_ch2_from_01262022070400__to__01262022070613.mp4', 'videos/vealnum_6715_ch1_from_01252022185900__to__01252022192350.mp4', 'videos/vealnum_6715_ch1_from_01252022204200__to__01252022205331.mp4', 'videos/vealnum_6684_ch2_from_01272022090800__to__01272022091205.mp4', 'videos/vealnum_6685_ch2_from_01252022091300__to__01252022091524.mp4', 'videos/vealnum_6694_ch1_from_01262022070200__to__01262022070343.mp4', 'videos/vealnum_6331_ch1_from_01252022173600__to__01252022174220.mp4', 'vide

MoviePy - Done.
Moviepy - Writing video videos/vealnum_6707_ch1_from_01252022090800__to__01252022091731.mp4



Moviepy - Done !
Moviepy - video ready videos/vealnum_6707_ch1_from_01252022090800__to__01252022091731.mp4
******created videos/vealnum_6707_ch1_from_01252022090800__to__01252022091731
hi ../../sante_veau/videos/2022_02_02/N862A6_ch1_main_20220125090000_20220125093000.mp4
True 2022-01-25 09:01:00 2022-01-25 09:00:00
** 60.0 601.0 the rest is -1199.0
we crop
Moviepy - Building video videos/vealnum_6700_ch1_from_01252022090100__to__01252022091001.mp4.
MoviePy - Writing audio in vealnum_6700_ch1_from_01252022090100__to__01252022091001TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videos/vealnum_6700_ch1_from_01252022090100__to__01252022091001.mp4



t:  30%|██▉       | 322/1082 [00:47<01:53,  6.70it/s, now=None]

In [1]:
!pip --isolated install deeplabcut

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/nix/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/nix/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Ignoring pip: markers 'python_version < "3"' don't match your environment
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/deeplabcut-2.2.0.6+computecanada-py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/nix/avx2/tables-3.6.1+computecanada-cp37-cp37m-linux_x86_64.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/tf_slim-1.1.0+computecanada-py2.py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/networkx-2.7+computecanada-py3-none-any.whl
ERROR: Package 'networkx' requires a different Python: 3.7.4 not in '>=3.8'
